# Logistic Regression
* LR的变量筛选主要从稳定性、重要性、相关性和解释性这几个维度下手。
  因为没有更多跨时代和跨渠道的样本，稳定性暂时没有考虑
## 特征筛选
* 通过IV值来衡量变量的重要性。IV < 0.02说明单变量区分能力较弱，可以筛选掉。

In [3]:
def discretize(data, columns_continous, quantiles):
    '''
    等频分箱函数
    input:
    data: dataframe, 原始变量数据
    columns_continous: list， 连续性变量列表
    quantiles: list, 等频分箱的分位点列表
    return:
    data_bin: dataframe, 分箱后的数据，每箱为字符型
    '''
    data_bin = data.copy()
    columns_cate = [column for column in data_bin.columns if column not in columns_continous]
    for column in columns_continous:
        X = data_bin[column].copy()
        for i in range(len(quantiles)-1):
            left = X.quantile(quantiles[i])
            right = X.quantile(quantiles[i+1])
            if i<len(quantiles)-2:
                group = '[' + str(left) + ',' + str(right) + ')'
                data_bin[column].iloc[np.where((X>=left)&(X<right))] = group
            if i==len(quantiles)-2:
                group = '[' + str(left) + ',' + str(right) + ']'
                data_bin[column].iloc[np.where((X>=left)&(X<=right))] = group
        data_bin[column].fillna('nan', inplace=True)
    for column in columns_cate:
        data_bin[column] = data_bin[column].astype(str)
    return data_bin

In [4]:
def woe_iv_calc(data_bin, y):
    '''
    计算WOE和IV函数
    input:
    data_bin: dataframe，分箱后的函数
    y: series，目标变量，值为0或1
    return:
    data_woe: dataframe，WOE映射后的数据
    map_woe: dict, key为变量名，value为每个箱对应的WOE值
    map_iv: dict, key为变量名，value为每个箱对应的IV值
    '''
    data_woe = data_bin.copy()
    map_woe, map_iv = {}, {}
    for column in data_woe.columns:
        cross = pd.crosstab(data_woe[column], y)
        cross[cross==0] = 1 # 解决分母为0的问题
        cross = cross/cross.sum(axis=0)
        woe = np.log(cross[0]/cross[1])
        iv = ((cross[0]-cross[1])*np.log(cross[0]/cross[1])).sum()
        map_woe[column] = dict(woe)
        map_iv[column] = iv
        data_woe[column] = data_woe[column].map(dict(woe))
        return data_woe, map_woe, map_iv

In [5]:
# # 调用函数
# X_columns = data_lr.columns[2:]
# Y_columns = 'TARGET'
# columns_continuous = eda_stat[eda_stat['count_unique'] > 10].index.tolist()
# columns_continuous = [column for column in columns_continuous if column != 'SK_ID_CURR']
# quantiles = [0.1*i for i in range(11)]
# data_bin = discretize(data_lr[X_columns], columns_continous, quantiles)
# data_woe, map_woe, map_iv = woe_iv_calc(data_bin, data_lr[Y_columns])

* 变量的相关性包括单变量之间的相关性和多变量之间的多重共线性，这两类相关性过高都会导致线性模型系数不置信。

* 相关性通常利用皮尔逊相关系数来衡量，如果系数大于0.8，则认为两个变量强相关，则只需要保留其中IV值较高的那个变量。

* 多重共线性主要通过VIF来计算，VIF > 10说明该变量和其它变量存在线性关系，应该将该变量筛掉

In [8]:
# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # 计算相关性
# data_cor = data_lr[columns_select].corr().abs()
# data_cor_lower = pd.DataFrame(np.tril(data_cor), index=data_cor.index, columns=data_cor.columns)

# columns_drop = []
# for column in data_cor_lower:
#     data_cor_select = pd.DataFrame(data_cor_lower.ix[(data_cor_lower[column]>0.8)&(data_cor_lower[column]<1), column])
#     if len(data_cor_select) > 0:
#         data_cor_select = pd.DataFrame(data=data_cor_select.columns.tolist() + data_cor_select.index.tolist(), columns=['column_name'])
#         data_cor_select['IV'] = data_cor_select['column_name'] = map(map_iv)
#         data_cor_select = data_cor_select.sort_values(by='IV', ascending=False)
#         columns_crop = columns_crop + data_cor_select['column_name'].tolist()[1:]

# columns_select = [column for column in columns_select if column not in columns_drop]
# data_lr = data_lr[['SK_ID_CURR', 'TARGET'] + columns_select]

In [9]:
# # 计算多重共线性
# data_vif = data_lr.ix[:, 2:].copy()
# data_vif = sm.add_constant(data_vif)
# data_vif = data_vif.replace([np.nan, np.inf], -9999)

# vif_select = pd.DataFrame(data=data_vif.columns, columns=['column_name'])
# vif_select['VIF'] = [variance_inflation_factor(data_vif.values, i) for i in range(data_vif.shape[1])]

# columns_select = vif_select.ix[vif_select['VIF']<10, 'column_name'].tolist()
# columns_select = [column for column in columns_select if 'const' not in column]
# data_lr = data_lr[['SK_ID_CURR', 'TARGET'] + columns_select]

In [10]:
def woe_plot(map_woe, close=True, show_last=True):
    '''
    WOE值分布图
    input:
    map_woe: dict, key为变量名， value为每箱对应的WOE值，建议每箱预先排序方便观察单调性
    close: bool，是否打印WOE值分布图，默认为True
    show_last: bool, 是否只保留最后一个变量的WOE值分布图，默认为True
    
    return:
    result: dict, key为变量名，value为每个变量的WOE值分布图
    '''
    result = {}
    for i, feature in enumerate(map_woe):
        data = pd.Series(map_woe[feature])
        data.index.name = ''
        data.name = ''
        fig = plt.figure()
        ax = fig.add_subplot(111)
        data.plot(kind='bar', ax=ax)
        ax.set_xlabel('变量分箱')
        ax.set_ylabel('WOE值')
        ax.set_title('%s' %feature)
        result[feature] = fig
        if close and show_last and i<len(map_woe)-1:
            plt.clost('all')
    return result

In [11]:
# # 训练模型
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import LogisticRegression


# data_lr[columns_select] = data_woe[columns_select]
# X_columns = data_lr.columns[2:]
# Y_columns = 'TARGET'
# X_train, X_test, y_train, y_test = train_test_split(data_lr[X_columns], data_lr[Y_columns], test_size=0.3, random_state=0)
# tuned_parameters = [{'penalty':['l1', '12'], 'C':[0.001, 0.01, 0.1, 1, 10]}]
# clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5, scoring='roc_auc')
# clf.fit(X_train, y_train)
# clf.best_params_
# lr = LogisticRegression(penalty='l2', C=0.1)
# lr_clf = lf.fit(X_train, y_train)

# XGBoost

* 筛选特征的方法：基于XGBoost输出的feature_importance对变量进行重要性排序，根据不同的重要性阈值进行筛选，观察每组变量筛选后模型在验证集或者测试集上的效果，选择保持模型效果且入模特征数量较少的阈值



In [1]:
# from xgboost import XGBClassifier
# import xgboost as xgb
# from sklearn.feature_selection import SelectFromModel
# from sklearn.metrics import roc_curve, auc, roc_auc_score


# X_columns = data_xgb.columns[2:]
# Y_columns = ['TARGET']

# X_train, X_test, y_train, y_test = train_test_split(data_xgb[X_columns], data_xgb[Y_columns], test_size=0.3, random_state=0)
# X_matrix_train = X_train.as_matrix(columns=None)
# Y_matrix_train = y_train.as_matrix(columns=None)
# X_matrix_test = X_test.as_matrix(columns=None)
# Y_matrix_test = y_test.as_matrix(columns=None)
# xgb_clf = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, min_child_weight=10, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', seed=0, n_jobs=-1)

# print(datetime.now())
# xgb_clf.fit(X_matrix_train, Y_matrix_train, eval_metric='auc')
# y_pred = xgb_clf.predict_proba(X_matrix_test)[:, 1]
# fpr, tpr, threshold = roc_curve(Y_matrix_test, y_pred)
# roc_auc = auc(fpr, tpr)
# print('test auc: %f' %roc_auc)
# print(datetime.now())

# threshold = np.sort(xgb_clf.feature_importances_)
# print(datetime.now())
# for thresh in threshold[400:700:10]:
#     selection = SelectFromModel(xgb_clf, threshold=thresh, prefit=True)
#     X_train_selection = selection.transform(X_matrix_train)
#     selection_model = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, min_child_weight=10, subsample=0.8, colsample_bytree=0.8, 
#                                     objective='binary:logistic', seed=0, n_jobs=-1)
#     selection_model.fit(X_train_selection, Y_matrix_train, eval_metric='auc')
#     X_test_selection = selection.transform(X_matrix_test)
#     y_pred = selection_model.predict_proba(X_test_selection)[:, 1]
#     fpr, tpr, threshold = roc_curve(Y_matrix_test, y_pred)
#     roc_auc = auc(fpr, tpr)
#     print('thresh = %.4f, n=%d, test auc: %f' % (thresh, X_train_selection.shape[1], roc_auc))
# print(datetime.now())

* 贝叶斯优化的优点在于，每次调参都会考虑上一组超参数的信息，迭代次数少，调参效率更高
* 针对非凸的损失函数，贝叶斯优化也不容易陷入局部最优，能够得到全局最优的超参数组合

In [5]:
# # 利用hyperopt对XGBoost模型调参
# from hyperopt import fmin, tpe, hp, partial


# X_columns = data_xgb.columns[2:]
# Y_columns = ['TARGET']

# X_train, X_test, y_train, y_test = train_test_split(data_xgb[X_columns], data_xgb[Y_columns], test_size=0.3, random_state=0)
# X_matrix_train = X_train.as_matrix(columns=None)
# Y_matrix_train = y_train.as_matrix(columns=None)
# X_matrix_test = X_test.as_matrix(columns=None)
# Y_matrix_test = y_test.as_matrix(columns=None)
# eval_set = [(X_matrix_train, Y_matrix_train), (X_matrix_test, Y_matrix_test)]

# # 定义超参数空间
# space = {
#             'max_depth' : hp.randint('max_depth', 2), 
#             'n_estimators' : hp.randint('n_estimators', 1500), 
#             'learning_rate' : hp.randint('learning_rate', 50), 
#             'min_child_weight' : hp.randint('min_child_weight', 100),
#             'subsample' : hp.randint('subsample', 5), 
#             'colsample_bytree' : hp.randint('colsample_bytree', 5), 
#             'alpha' : hp.randint('alpha', 1000), 
#             'lambda' : hp.randint('lambda', 1000)
#         }

# def argsDict_transform(argsDict, isPrint=False):
#     argsDict['max_depth'] = argsDict['max_depth'] + 4
#     argsDict['n_estimators'] = argsDict['n_estimators'] + 500
#     argsDict['learning_rate'] = argsDict['learning_rate'] * 0.01 + 0.01
#     argsDict['min_child_weight'] = argsDict['min_child_weight'] + 1
#     argsDict['subsample'] = argsDict['subsample'] * 0.1 + 0.5
#     argsDict['colsample_bytree'] = argsDict['colsample_bytree'] * 0.1 + 0.5
#     argsDict['alpha'] = argsDict['alpha'] * 0.01
#     argsDict['lambda'] = argsDict['lambda'] * 0.01
    
#     if isPrint:
#         print(argsDict)
    
#     return argsDict

# # 定义metric
# def aug_neg(y_pred, dtrain):
#     y_true = dtrain.get_label()
#     fpr, tpr, thresholds = roc_curve(y_true, y_pred)
#     auc_neg = -auc(fpr, tpr)
#     return 'auc_neg', auc_neg

# # 定义训练过程
# def xgboost_factory(argsDict):
#     argsDict = argsDict_transform(argsDict)
#     params = {
#                 'max_depth' : argsDict['max_depth'], 
#                 'n_estimators' : argsDict['n_estimators'], 
#                 'learning_rate' : argsDict['learning_rate'], 
#                 'min_child_weight' : argsDict['min_child_weight'],
#                 'subsample' : argsDict['subsample'], 
#                 'colsample_bytree' : argsDict['colsample_bytree'], 
#                 'objective' : 'binary:logistic',
#                 'silent' : True,
#                 'alpha' : argsDict['max_depth'], 
#                 'lambda' : argsDict['max_depth'],
#                 'seed' : 0
#             }
#     xgb_clf = XGBClassifier(**params)
#     xgb_clf.fit(X_matrix_train, Y_matrix_train, eval_set=eval_set, eval_metric=auc_neg, early_stopping_rounds=100, verbose=10)
#     return get_transformer_score(xgb_clf)

# def get_transformer_score(transformer):
#     xgb_clf = transformer
#     y_pred = xgb_clf.predict_proba(X_matrix_test, ntree_limit=xgb_clf.best_ntree_limit)[:, 1]
#     fpr, tpr, threshold = roc_curve(Y_matrix_test, y_pred)
#     return -auc(fpr, tpr)

In [6]:
# # 搜索最优超参数组合

# algo = partial(tpe.suggest, n_startup_jobs=1)
# best = fmin(xgboost_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)
# argsDict = argsDict_transform(best)
# params = {
#             'max_depth' : argsDict['max_depth'], 
#             'n_estimators' : argsDict['n_estimators'], 
#             'learning_rate' : argsDict['learning_rate'], 
#             'min_child_weight' : argsDict['min_child_weight'],
#             'subsample' : argsDict['subsample'], 
#             'colsample_bytree' : argsDict['colsample_bytree'], 
#             'objective' : 'binary:logistic',
#             'silent' : True,
#             'alpha' : argsDict['max_depth'], 
#             'lambda' : argsDict['max_depth'],
#             'seed' : 0
#         }

# # 加入早停条件
# xgb_clf = XGBClassifier(**params)
# xgb_clf = xgb_clf.fit(X_matrix_train, Y_matrix_train, eval_set=eval_set, eval_metric=auc_neg, early_stopping_rounds=100, verbose=10)

利用SHAP值进行XGBoost模型白盒化，解释入模特征对于目标的贡献度和贡献方向

In [ ]:
import shap


explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer.shap_values(data_train[X_columns])
shap.summary_plot(shap_values, data_train[X_columns])
shap.summary_plot(shap_values, data_train[X_columns], plot_type='bar')